# Лабораторная работа №8

## Метод главных компонент

### Задание 1

In [2]:
from sklearn import datasets

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

1. Используя данные для цветков ириса произвести уменьшение размерности данных с помощью метода главных компонет. Реализовать собственный алгоритм, а также использовать встроенный.
2. Оценить степень потери данных. 
3. Выберите оптимальное количество компонент в новых данных. 
4. Протестируйте точность различных алгоритмов классификации на исходном датасете и на преобразованном с помощью метода главных компонент. Дайте подробные выводы о различных комбинациях числа компонет и алгоритмах. 

In [9]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [7]:
import numpy as np

class CustomPCA:
    def __init__(self, n_components:int):
        self.n_components = n_components

    def fit_transform(self, X):
        mean_vals = np.mean(X, axis=0)
        centered_data = X - mean_vals
        cov_matrix = np.cov(centered_data, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        sorted_indices = np.argsort(eigenvalues)[::-1]
        top_indices = sorted_indices[:self.n_components]
        pca_result = centered_data.dot(eigenvectors[:, top_indices])
        self.eigenvalues = eigenvalues
        self.top_indices = top_indices
        return pca_result

# Применение собственного алгоритма PCA к данным ириса
cust_pca = CustomPCA(2)
X_custom_pca = cust_pca.fit_transform(X)

In [10]:
for i in [1,2,3,4]:
    cust_pca = CustomPCA(i)
    X_custom_pca = cust_pca.fit_transform(X)
    explained_variance_ratio_custom = np.sum(cust_pca.eigenvalues[cust_pca.top_indices]) / np.sum(cust_pca.eigenvalues)
    print(f"Собственный алгоритм PCA: Объясненная дисперсия при {i} компонент - {explained_variance_ratio_custom:.4f}")

Собственный алгоритм PCA: Объясненная дисперсия при 1 компонент - 0.9246
Собственный алгоритм PCA: Объясненная дисперсия при 2 компонент - 0.9777
Собственный алгоритм PCA: Объясненная дисперсия при 3 компонент - 0.9948
Собственный алгоритм PCA: Объясненная дисперсия при 4 компонент - 1.0000


In [11]:
from sklearn.decomposition import PCA

for i in [1,2,3,4]:
    pca = PCA(n_components=i)
    X_builtin_pca = pca.fit_transform(X)
    explained_variance_ratio_builtin = np.sum(pca.explained_variance_ratio_)
    print(f"Встроенный PCA: Объясненная дисперсия при {i} компонент- {explained_variance_ratio_builtin:.4f}")


Встроенный PCA: Объясненная дисперсия при 1 компонент- 0.9246
Встроенный PCA: Объясненная дисперсия при 2 компонент- 0.9777
Встроенный PCA: Объясненная дисперсия при 3 компонент- 0.9948
Встроенный PCA: Объясненная дисперсия при 4 компонент- 1.0000


In [12]:
pca = PCA(n_components=2)
X_builtin_pca = pca.fit_transform(X)

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Исходные данные
lr_original = LogisticRegression()
knn_original = KNeighborsClassifier()
dt_original = DecisionTreeClassifier()

# Преобразованные данные
lr_pca = LogisticRegression()
knn_pca = KNeighborsClassifier()
dt_pca = DecisionTreeClassifier()

# Точность на исходных данных
acc_lr_original = cross_val_score(lr_original, X, y, cv=5, scoring='accuracy').mean()
acc_knn_original = cross_val_score(knn_original, X, y, cv=5, scoring='accuracy').mean()
acc_dt_original = cross_val_score(dt_original, X, y, cv=5, scoring='accuracy').mean()

# Точность на данных после PCA
acc_lr_pca = cross_val_score(lr_pca, X_builtin_pca, y, cv=5, scoring='accuracy').mean()
acc_knn_pca = cross_val_score(knn_pca, X_builtin_pca, y, cv=5, scoring='accuracy').mean()
acc_dt_pca = cross_val_score(dt_pca, X_builtin_pca, y, cv=5, scoring='accuracy').mean()

# Вывод результатов
print("Исходные данные:")
print(f"Logistic Regression: {acc_lr_original:.4f}")
print(f"K-Nearest Neighbors: {acc_knn_original:.4f}")
print(f"Decision Tree: {acc_dt_original:.4f}")
print("\nPCA данные:")
print(f"Logistic Regression: {acc_lr_pca:.4f}")
print(f"K-Nearest Neighbors: {acc_knn_pca:.4f}")
print(f"Decision Tree: {acc_dt_pca:.4f}")


/Users/natalliazzz/miniconda3/envs/ds_labs/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Исходные данные:
Logistic Regression: 0.9733
K-Nearest Neighbors: 0.9733
Decision Tree: 0.9533

PCA данные:
Logistic Regression: 0.9600
K-Nearest Neighbors: 0.9667
Decision Tree: 0.9200


### Задание 2

1. Используя данные о клиентах сотового оператора (telecom.csv) произвести уменьшение размерности данных с помощью метод главных компонет. Реализовать собственный алгоритм, а также использовать встроенный.
2. Оценить степень потери данных. 
3. Выберите оптимальное количетсво компонент в новых данных. 
4. Протестируйте точность различных алгоритмов классификации на исходном датасете и на преобразованном с помощью метода главных компонент. Дайте подробные выводы о различных комбинациях числа компонет и алгоритмах. 

### Задание 3

In [ ]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

1. Используя данные о рукописных цифрах произвести уменьшение размерности данных с помощью метода главных компонет.Реализовать собственный алгоритм, а также использовать встроенный.
2. Оценить степень потери данных. 
3. Выберите оптимальное колиечество компонент в новых данных. 
4. Протестируйте точность различных алгоритмов классификации на исходном датасете и на преобразованном с помощью метода главных компонент. Дайте подробные выводы о различных комбинациях числа компонет и алгоритмах. 

### Задание 4

Вычислениями подвердите связь сингулярного разложения матрицы с методом главных компонент. Приведите практический пример.

## Полезные ссылки

1. [Хорошая статья на русском о PCA](https://habr.com/ru/post/304214/)
2. [sklearn PCA doc](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
3. [Выбор количества компонент](https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/)
4. [PCA через сингулярное разложение](https://towardsdatascience.com/pca-and-svd-explained-with-numpy-5d13b0d2a4d8)